# Домашняя работа. Деревья решений.

## Полезная литература

- [Habrahabr: ODS деревья решений](https://habrahabr.ru/company/ods/blog/322534/#derevo-resheniy)
- [ВМК МГУ семинары по решающим деревьям](99-extra__ml-course-msu-Sem04_trees.pdf)
- [Sklearn Decision Trees](http://scikit-learn.org/stable/modules/tree.html)

## 1. Сравнение моделей деревьев

В этом блоке вы сравните разные конфигурации композиций деревьев:
- DecisionTree
- Bagging
- Bagging с другими настройками подбора признаков для разбиения
- RandomForest

Будем использовать [датасет с винишком](https://archive.ics.uci.edu/ml/datasets/wine+quality) - это задача то ли классификации то ли регресси - нужно предсказывать качество вина. Будем думать что это классификация.

![](https://upload.wikimedia.org/wikipedia/en/thumb/7/7c/Lulz_Security.svg/300px-Lulz_Security.svg.png)

### 1.1 Чтение данных (1 балла)

Данные лежат как обычно в `'./data/winequality-red.csv.gz'`.

- Прочитайте их с помощью pandas
- нарисуйте countplot целевого признака `quality`.
- Что вы думаете по поводу количества представителей каждого класса.
- Разбейте данные на X и y.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
uploaded = files.upload()


# Прочитайте файл
wine_data = pd.read_csv('winequality-red.csv', sep=';')

In [ ]:
# Построение countplot
plt.figure(figsize=(10, 6))
sns.countplot(data=wine_data, x='quality')
plt.title('Distribution of Wine Quality')
plt.show()

# Разделение на X и y
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

In [ ]:
'По поводу распределения классов: Обычно наблюдается несбалансированное распределение - больше всего вин среднего качества (5-6), меньше всего - очень высокого и очень низкого качества.'

### 1.2 Сравнение моделей (4 балла)

Задача классификации. Все признаки уже числовые. Значит можно пробовать просто все модели и выбрать лучшую. Так и поступим, сделайте кросс валидацию на 5 фолдах, используя `sklearn.model_selection.KFold` как аргумент у `cross_val_score`. Метрика качества будет `accuracy`.

Алгоритмы для тестирования:
- KNeighborsClassifier с 10 соседями
- KNeighborsClassifier с 10 соседями и масштабированием StandartScaler
- RidgeClassifier
- DecisionTreeClassifier 
- BaggingClassifier c 100 деревьев
- BaggingClassifier с 100 деревьев и каждое дерево обучается только по половине случайно выбранных признаков (см аргументы)
- RandomForestClassifier c 100 деревьев

Выведите среднее значение метрики качества для каждого из классификаторов. 

**hint**: каждый следующий алгоритм, будет показывать качество лучше, чем предыдущий. Если у вас не так - то что-то вы делаете неправильно. Везде зафиксируйте random_state=42.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline  # добавляем этот импорт

# Создаем KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Список моделей
models = {
    'KNN': KNeighborsClassifier(n_neighbors=10),
    'KNN+Scaling': Pipeline([('scaler', StandardScaler()), 
                           ('knn', KNeighborsClassifier(n_neighbors=10))]),
    'Ridge': RidgeClassifier(random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'Bagging': BaggingClassifier(n_estimators=100, random_state=42),
    'Bagging_half_features': BaggingClassifier(n_estimators=100, 
                                             max_features=0.5,
                                             random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Оценка моделей
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    print(f'{name}: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})')

### 1.3 Расуждения (8 баллов)

Ответьте на вопросы развернуто, можете полистать литературу:

- почему наблюдается значимая разница в качестве у KNeighborsClassifier с масштабированием и без
- почему масштабирование не важно для деревьев решений
- почему бэггинг на половине признаков для каждого дерева дал качество предсказания больше, чем на всех? (а он дал!)
- у какой модели наибольшей отклонение от среднего качества предсказаний? А почему??

In [ ]:
"Разница в качестве KNN с масштабированием и без объясняется тем, что KNN чувствителен к масштабу признаков. Без масштабирования признаки с большими значениями доминируют при расчете расстояний.
Деревья решений нечувствительны к масштабированию, так как они работают с порядком значений при разбиении, а не с абсолютными величинами.
Бэггинг на половине признаков дает лучшее качество из-за снижения корреляции между деревьями, что уменьшает переобучение.
Наибольшее отклонение обычно у одиночного дерева решений, так как оно наиболее чувствительно к изменениям в данных."

## 2 Переобучение и Ко

В последнем задании вы уже заметили, что случайный лес может вести себя немного нестабильно. В этом задании мы возьмем опять датасет MNIST(простите) и будем его решать деревьями. Почему мы взяли его? Потому что в нем фактически много разных признаков (значения пикселей в пространстве), а деревья строятся делая разбиения по признакам. Обычно на эти разбиения не обращают внимание, так как главное что тюнят - это глубина дереьвев, количество деревьев, а кучу других параметров обходят стороной, так как они "неясные". Попробуем прояснить их.

### 2.1 Загрузка датасета (1 балл)

Загрузите датасет с помощью функции `sklearn.datasets.load_digits`. В нем будут 64px картинки в векторной форме.

Нарисуйте первые 10 цифр в одной ячейке, чтобы было красиво.

In [ ]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import numpy as np

# Загрузка данных
digits = load_digits()
X, y = digits.data, digits.target

In [ ]:
# Визуализация первых 10 цифр
fig, axes = plt.subplots(1, 10, figsize=(20, 2))
for ax, image, label in zip(axes, digits.images[:10], digits.target[:10]):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r)
    ax.set_title(f'Digit: {label}')
plt.show()

### 2.2 Перебор классификаторов (3 балла)

В этом задании вам снова придется перебрать несколько классификаторов, но теперь мы обратим внимание на другие гиперпараметры и их влияние на качество классификации, кстати опять `accuracy`.

Сделайте кроссвалидацию на 10 фолдах, указав `cv=10` для следующих классификаторов:

- DecisionTreeClassifier с параметрами по-умолчанию
- BaggingClassifier с 100 деревьвев
- BaggingClassifier с 100 деревьев, НО с ограничением на максимальное количество признаков, участвующих при обучении каждого из деревьев в $\sqrt{N}$, где $N$ - это число признаков.
- BaggingClassifier с 100 деревьев, НО с ограничением на количество признаков участвующих в разбиении для каждого из деревьев в $\sqrt{N}$, где $N$ - это число признаков. Это отличается от предыдущей модели тем, где ограничивается `max_features`. Читайте документацию :trollface:
- обычный случайный лес со 100 деревьями

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import numpy as np

n_features = X.shape[1]
max_features = int(np.sqrt(n_features))

classifiers = {
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'Bagging': BaggingClassifier(n_estimators=100, random_state=42),
    'Bagging_sqrt_features': BaggingClassifier(
        n_estimators=100,
        max_features=max_features,
        random_state=42
    ),
    'Bagging_sqrt_split': BaggingClassifier(
        estimator=DecisionTreeClassifier(max_features=max_features),
        n_estimators=100,
        random_state=42
    ),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, clf in classifiers.items():
    scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print(f'{name}: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})')

### 2.3 В чём разница? (3 балла)

Ответье на вопрос: 

Странно то как? Почему ограничение на количество признаков в разбиении дерева и ограничение в количестве признаков для построения каждого дерева в BaggingClasifier дало СОВСЕМ разный результат в качестве предсказания? В чем магия?

### 2.4 Количество деревьев (2 балла)

Сделайте перебор количества деревьев для `RandomForestClassifier`. Сохраните качества кросс валидации на 10 фолдах для `[1,5,10,15,50,100,150,200,300]` количества деревьев. Нарисуйте график, где по оси x - количество деревьев, а по оси y - качество. При каком количестве деревьев получается самое хорошее качество?

In [ ]:
n_trees = [1, 5, 10, 15, 50, 100, 150, 200, 300]
scores = []

for n in n_trees:
    rf = RandomForestClassifier(n_estimators=n, random_state=42)
    score = cross_val_score(rf, X, y, cv=10, scoring='accuracy').mean()
    scores.append(score)

plt.figure(figsize=(10, 6))
plt.plot(n_trees, scores, '-o')
plt.xlabel('Number of trees')
plt.ylabel('Accuracy')
plt.title('Random Forest Performance vs Number of Trees')
plt.grid(True)
plt.show()

### 2.5 Количество признаков  (2 балла)

Переберите теперь максимальное количество признаков для `RandomForestClassifier` на 100 деревьях, от 1 до 64 с шагом 5. Постройте график качества по кроссвалидации на 10 фолдах.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Перебор максимального количества признаков для RandomForestClassifier на 100 деревьях
max_features = list(range(1, 65, 5))
scores = []
for max_feature in max_features:
    algorithm = RandomForestClassifier(n_estimators=100, max_features=max_feature)
    scores_max_feature = cross_val_score(algorithm, digits.data, digits.target, cv=10)
    scores.append(scores_max_feature.mean())

# Нарисуйте график качества по кроссвалидации на 10 фолдах
plt.plot(max_features, scores)
plt.xlabel('Максимальное количество признаков')
plt.ylabel('Качество')
plt.title('Качество RandomForestClassifier по максимальному количеству признаков')
plt.show()

### 2.5 Вопросы по RandomForest (8 баллов)

Ответьте на вопросы:

- Что происходит с ростом числа деревьев у случайного леса. Можно ли просто всегда брать 5000 деревьев и быть счастливым?
- Как зависит качество предсказания в дереве в зависимости от max_features?
- Почему качество зависит от max_features?
- Как глубина деревьев влияет на качество случайного леса?

С ростом числа деревьев:
Качество обычно улучшается до определенного момента
После этого улучшение незначительное
Брать 5000 деревьев не всегда оптимально из-за:
Увеличения времени обучения и предсказания
Увеличения требований к памяти
Незначительного прироста в качестве после определенного порога
Зависимость от max_features:
При малых значениях - модель может упустить важные паттерны
При больших значениях - может быть переобучение
Оптимальное значение обычно около sqrt(n_features)
max_features влияет на качество потому что:
Контролирует разнообразие деревьев
Влияет на способность модели находить важные признаки
Балансирует между обобщающей способностью и точностью
Глубина деревьев влияет:
Малая глубина - недообучение
Большая глубина - переобучение
Оптимальная глубина зависит от сложности данных и количества примеров

![](https://habrastorage.org/web/ad8/366/a44/ad8366a4469346c6b2e1306495b05d1a.jpg)